# **Transaction Agent powered by FloTorch:**

**Problem it solves & for whom**: Helps customers make sense of their spending patterns by automatically categorizing transactions (e.g., groceries, travel, bills) and detecting unusual or fraudulent activity. It’s useful for individuals and financial institutions that want real-time spending insights and fraud alerts.

**Input:** Bank statements or credit card statements in PDF or CSV format are given as input.

**Output:** The output would be a downloadable report containing all the insights and visualizations.

**Why it matters:** Most people struggle to track where their money goes each month. Automating categorization and anomaly detection saves time, prevents overspending, and increases financial security through early fraud detection.

Install all required dependencies for the notebook. It includes data analysis and visualization tools (pandas, matplotlib, plotly), the flotorch framework for AI agent evaluation, and libraries for PDF generation and text extraction (fpdf2, pdfplumber).

In [1]:
# Pandas: data manipulation and analysis
# Matplotlib: static plotting library
# Plotly: interactive visualizations
!pip install pandas matplotlib plotly --quiet

# Flotorch (with "strands" extras): AI agent evaluation and tracing framework
# Version pinned to beta 2.1.0b1 for compatibility
!pip install flotorch[strands]==2.1.0b1

# fpdf2: library for generating PDFs programmatically
# pdfplumber: tool for extracting text and tables from PDFs
!pip install -q fpdf2 pdfplumber


Imports essential modules: userdata from Google Colab for securely handling credentials, and FlotorchStrandsAgent from the Flotorch library to define and manage intelligent agents used in the workflow.

In [2]:
# Import utilities for securely accessing stored secrets or tokens in Google Colab
# (e.g., API keys or project credentials)
from google.colab import userdata

# Import the FlotorchStrandsAgent class from the flotorch.strands module
# This class enables creation and interaction with intelligent agent workflows
# that can be instrumented and evaluated via Flotorch
from flotorch.strands.agent import FlotorchStrandsAgent


Initializes the **Transaction Agent client** using the Flotorch SDK. It connects securely to the Flotorch Gateway via the provided API key and base URL, enabling interaction with the “transaction-agent” for downstream tasks such as transaction categorization and analysis.


In [3]:
# Initialize a FlotorchStrandsAgent instance for the "Transaction Agent"

transaction_agent_client = FlotorchStrandsAgent(
    agent_name="transaction-agent",             # Unique name identifying this agent
    api_key=userdata.get('flotorch_api_key'),   # Securely retrieves the Flotorch API key stored in Colab's userdata
    base_url="https://gateway.flotorch.cloud"   # Base URL for the Flotorch Gateway where the agent communicates
)


2025-11-04 19:15:24 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 19:15:24 - flotorch.sdk.llm - INFO - FlotorchLLM initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 19:15:24 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


2025-11-04 19:15:24 - flotorch.strands.llm - INFO - FlotorchStrandsModel initialized (model_id=flotorch/use-case-building:latest, base_url=https://gateway.flotorch.cloud)


Fetches the active Transaction Agent instance from the Flotorch Gateway. This is already created and defined in the backend. Once retrieved, the agent can be invoked to analyze and categorize financial transactions in subsequent steps.

In [4]:
# Retrieve the deployed or configured "Transaction Agent" instance
# This call initializes a ready-to-use agent object that can process inputs
# and perform tasks such as transaction analysis or categorization

transaction_agent = transaction_agent_client.get_agent()


Handles file uploads and data loading.



*  If the user uploads a CSV, it is directly read into a pandas DataFrame.  
*  If PDFs are uploaded, up to 12 files are processed with pdfplumber to extract tabular data from each page.
All extracted tables are merged into one dataset, saved as merged_from_pdfs.csv, and loaded into the variable df for further processing.

In [5]:
# === FILE UPLOAD & LOAD (PDF or CSV) ==========================================
# This cell handles user uploads of either PDF or CSV files and loads them into a pandas DataFrame.
# If PDFs are provided, tables are extracted from up to 12 PDF files and merged into one CSV.
# The final DataFrame is stored as `df`.

import io
import os
import sys
import pandas as pd

# Check if running in Google Colab environment
try:
    from google.colab import files
except Exception as e:
    raise RuntimeError("This cell is designed for Google Colab. `google.colab` not available.") from e

# Ask user for input type (pdf or csv)
kind = input("Are you uploading pdf or csv? (pdf/csv): ").strip().lower()
if kind not in {"pdf", "csv"}:
    raise ValueError("Please type exactly 'pdf' or 'csv'.")

# =========================== CSV HANDLING ===========================
if kind == "csv":
    print("Upload a CSV file…")
    uploaded = files.upload()  # Prompt user to select file
    if not uploaded:
        raise RuntimeError("No file uploaded.")
    # Pick first uploaded file
    fname = next(iter(uploaded.keys()))
    if not fname.lower().endswith(".csv"):
        raise ValueError(f"Uploaded file '{fname}' is not a .csv")
    file_path = f"/content/{fname}"
    # Write to disk
    with open(file_path, "wb") as f:
        f.write(uploaded[fname])
    # Load into DataFrame
    df = pd.read_csv(file_path)
    print(f"Loaded CSV: {file_path} with {len(df):,} rows.")

# =========================== PDF HANDLING ===========================
else:
    print("Upload up to 12 PDF files…")
    uploaded = files.upload()
    if not uploaded:
        raise RuntimeError("No files uploaded.")
    pdf_files = [name for name in uploaded.keys() if name.lower().endswith(".pdf")]
    if not pdf_files:
        raise ValueError("No PDF files detected in the upload.")
    if len(pdf_files) > 12:
        raise ValueError(f"You uploaded {len(pdf_files)} PDFs. Please upload at most 12.")

    # Save each uploaded PDF to disk
    pdf_paths = []
    for name in pdf_files:
        path = f"/content/{name}"
        with open(path, "wb") as f:
            f.write(uploaded[name])
        pdf_paths.append(path)

    # Ensure pdfplumber is installed for table extraction
    try:
        import pdfplumber  # noqa
    except ImportError:
        print("Installing pdfplumber…")
        !pip -q install pdfplumber
        import pdfplumber  # noqa

    # Extract tables from each PDF page and collect into list
    all_tables = []
    import pdfplumber
    for p in pdf_paths:
        print(f"Extracting tables from {os.path.basename(p)} …")
        with pdfplumber.open(p) as pdf:
            for page_idx, page in enumerate(pdf.pages):
                try:
                    # Attempt structured extraction using line-based detection
                    tables = page.extract_tables(table_settings={
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                        "intersection_tolerance": 5,
                    })
                except Exception:
                    # Fallback to default extraction if above fails
                    tables = page.extract_tables()
                for t in tables or []:
                    if not t or len(t) < 1:
                        continue
                    # Identify headers and clean empty ones
                    header = t[0]
                    body = t[1:] if len(t) > 1 else []
                    header_clean = [
                        h if (h is not None and str(h).strip()) else f"col_{i}"
                        for i, h in enumerate(header)
                    ]
                    df_tbl = pd.DataFrame(body, columns=header_clean)
                    # Keep table if it has meaningful data
                    if df_tbl.shape[1] >= 2 and df_tbl.shape[0] >= 1:
                        all_tables.append(df_tbl)

    # Check extracted results
    if not all_tables:
        raise RuntimeError("No tables detected in the uploaded PDFs. Please ensure your PDFs contain tabular data.")

    # Merge all extracted tables into one DataFrame
    merged_df = pd.concat(all_tables, ignore_index=True)

    # Save to CSV for reproducibility
    file_path = "/content/merged_from_pdfs.csv"
    merged_df.to_csv(file_path, index=False)

    # Assign merged data to `df`
    df = merged_df
    print(f"Merged {len(pdf_paths)} PDFs → {file_path} with {len(df):,} rows.")


2025-11-04 19:15:25 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.


2025-11-04 19:15:25 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.
Are you uploading pdf or csv? (pdf/csv): csv
Upload a CSV file…


Saving personal_transactions.csv to personal_transactions (1).csv
Loaded CSV: /content/personal_transactions (1).csv with 806 rows.


Cleans and standardizes the dataset. It ensures that every file has consistent column names for Date, Amount, and Description, even if the uploaded data used different labels. After normalizing, it previews the first 10 rows of the cleaned DataFrame to confirm successful parsing.

In [6]:
# === NORMALIZE COLUMNS & PREVIEW =============================================
# Soft-parse a date column, normalize amount column name, and ensure a description column exists.
# Expects `df` and `file_path` to already be defined by the previous upload cell.

import pandas as pd
from IPython.display import display

# --- Identify and parse date columns ---
# Look for a column that matches common date naming patterns
date_cols = [c for c in df.columns if str(c).lower() in ["date", "posted_date", "transaction_date", "datetime"]]
if date_cols:
    # Convert recognized date column to datetime format (coercing invalid entries to NaT)
    df[date_cols[0]] = pd.to_datetime(df[date_cols[0]], errors="coerce")
    # Rename first matching column to standard "Date"
    df.rename(columns={date_cols[0]: "Date"}, inplace=True)
else:
    # If no date column exists, create one filled with NaT to maintain structure
    df["Date"] = pd.NaT  # absent

# --- Normalize amount column ---
amt_col = None
# Search for a column name representing transaction amount
for c in df.columns:
    if str(c).lower() in ["amount", "amt", "value", "transaction_amount"]:
        amt_col = c
        break
# Raise an error if no recognizable amount column is found
if amt_col is None:
    raise ValueError("Could not find an amount-like column (e.g., Amount). "
                     "Please rename your amount column to 'Amount' and re-run.")

# Standardize amount column name to "Amount"
df.rename(columns={amt_col: "Amount"}, inplace=True)

# --- Ensure a description/merchant column exists ---
desc_col = None
# Try finding columns that likely contain text descriptions or merchant names
for c in df.columns:
    if str(c).lower() in ["description", "merchant", "payee", "narration", "memo"]:
        desc_col = c
        break
if desc_col is None:
    # If none exist, create a fallback "Description" column with row numbers
    df["Description"] = "Row-" + (df.reset_index().index + 1).astype(str)
    desc_col = "Description"

# --- Display summary and preview ---
print("Loaded/Prepared data from:", file_path)
print("Columns normalized to include: ['Date', 'Amount', 'Description', ...]")
display(df.head(10))  # Show first 10 rows for verification


Loaded/Prepared data from: /content/personal_transactions (1).csv
Columns normalized to include: ['Date', 'Amount', 'Description', ...]


,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card
5,2018-01-05,American Tavern,25.85,debit,Restaurants,Silver Card
6,2018-01-06,Hardware Store,18.45,debit,Home Improvement,Silver Card
7,2018-01-08,Gas Company,45.00,debit,Utilities,Checking
8,2018-01-08,Hardware Store,15.38,debit,Home Improvement,Silver Card
9,2018-01-09,Spotify,10.69,debit,Music,Platinum Card


Prepare a compact text sample of the uploaded dataset for the language model.
It selects the first 80 rows (or fewer if the file is smaller) and converts them into CSV-formatted text, making the data concise and structured for LLM-based processing.

In [7]:
# df is already loaded from your uploaded CSV

# Limit the number of rows passed to the model to avoid excessive token usage
# (Here we cap at 80 rows to stay within safe token limits)
n_rows = min(80, len(df))  # keep tokens under control

# Extract the top n_rows from the dataframe to create a manageable sample
sample_df = df.head(n_rows)

# Convert the sample dataframe into a CSV-formatted text string
# This makes the data compact, structured, and easier for an LLM to parse
sample_text = sample_df.to_csv(index=False)

# Print confirmation message for user visibility
print("Using first", n_rows, "rows as LLM context.")

Using first 80 rows as LLM context.


Construct the prompt text that will be sent to the Transaction Agent. It embeds the sampled transaction CSV and clearly instructs the model to categorize transactions, detect recurring patterns, and flag anomalies, returning a short, human-readable bullet summary.

In [8]:
# Build a natural-language prompt for the Transaction Agent.
# This prompt includes:
# - A CSV-style sample of the user's transactions (`sample_text`)
# - Clear tasks: categorize, detect recurring patterns, and flag anomalies
# - Instructions on how to format the response (short, human-readable bullets, no JSON)

tx_prompt = f"""
You are the Transaction Agent.

Here are sample transactions (CSV):
---
{sample_text}
---

Tasks:
1) Assign a simple category per row (e.g., groceries, utilities, rent, transit, dining, shopping, subscriptions, income, transfers, fees, misc).
2) Detect recurring transactions (describe pattern: merchant/description + periodicity).
3) Flag basic anomalies (unusually large spend or out-of-pattern merchants). Keep it simple.

Return:
- A short bullet summary for (1), (2), (3).
- Keep it human-readable (no JSON).
"""


Configure logging behavior to keep notebook output clean.
It removes any existing log handlers from Flotorch and Strands modules, disables log propagation, and reduces verbosity to the WARNING level — ensuring that only essential messages appear during execution.

In [9]:
import logging

# Remove any existing Flotorch or Strands log handlers
# This prevents duplicate log messages or excessive verbosity in Colab output
for name in ["flotorch", "strands"]:
    logger = logging.getLogger(name)
    logger.handlers.clear()   # Clear existing handlers
    logger.propagate = False  # Prevent propagation to the root logger

# Optionally reduce log verbosity for cleaner output
# Setting both Flotorch and Strands modules to WARNING hides debug/info logs
logging.getLogger("flotorch").setLevel(logging.WARNING)
logging.getLogger("strands").setLevel(logging.WARNING)


Invoke the Transaction Agent with the prepared prompt. The agent analyzes the uploaded transactions, categorizes them, identifies recurring patterns, and flags any anomalies, returning a concise, human-readable summary stored in the variable response.

In [10]:
# Send the constructed prompt (tx_prompt) to the Transaction Agent
# The agent processes the sample transactions and performs:
#   1. Categorization of each transaction
#   2. Detection of recurring patterns
#   3. Identification of anomalies or unusual spending
# The result is stored in `response` for display or further analysis.

response = transaction_agent(tx_prompt)


## Transaction Analysis Summary

### 1) Category Assignment
The transactions are already well-categorized in the data. Here's a simplified mapping:
- **Income**: Biweekly Paycheck ($2000)
- **Housing**: Mortgage Payment ($1247.44), utilities (Gas/Power/Water/Internet)
- **Dining**: Various restaurants, fast food, coffee shops
- **Shopping**: Amazon purchases, hardware store
- **Subscriptions**: Netflix ($11.76), Spotify ($10.69)
- **Transportation**: Shell, BP, Gas Station fuel purchases
- **Groceries**: Grocery Store purchases
- **Transfers**: Credit Card Payments (both credits and debits)
- **Personal**: Barbershop ($30)

### 2) Recurring Transactions Detected
- **Biweekly Paycheck**: $2000 every ~2 weeks (Jan 12, 19; Feb 2, 16; Mar 2)
- **Monthly Mortgage**: $1247.44 on 2nd of each month (Jan 2, Feb 2, Mar 2)
- **Monthly Netflix**: $11.76 around 4th of each month (Jan 4, Feb 4, Mar 4)
- **Monthly Spotify**: $10.69 around 9th of each month (Jan 9, Feb 9)
- **Monthly Phone**: ~$89 aro

Generate a downloadable PDF report from the Transaction Agent’s response.
It extracts and cleans the text output, formats section headers and bullet points, and uses the fpdf2 library to build a structured “Transaction Analysis Report.” The completed PDF is stored in memory and displayed as a one-click download link inside Colab.

In [11]:
from fpdf import FPDF
from fpdf.enums import XPos, YPos
from IPython.display import HTML, display
from io import BytesIO
import base64, re

# --- Extract clean text from response ---
# The response from the Transaction Agent is typically a dictionary with the content field.
# This section extracts the agent's textual output safely, handling both dict and string cases.
if isinstance(response, dict) and "content" in response:
    tx_result = response["content"][0]["text"]
else:
    tx_result = str(response)

# --- Clean text ---
# Remove markdown-like symbols and unnecessary formatting characters.
# Then, split into non-empty lines for easier iteration when writing to PDF.
clean_text = re.sub(r"[*#`_]+", "", tx_result)
lines = [line.strip() for line in clean_text.splitlines() if line.strip()]

# --- Create the PDF ---
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)

# Add title section
pdf.set_font("Helvetica", style="B", size=16)
pdf.cell(0, 10, "Transaction Analysis Report",
         new_x=XPos.LMARGIN, new_y=YPos.NEXT, align="C")
pdf.ln(6)

# Add introduction text
pdf.set_font("Helvetica", size=12)
pdf.multi_cell(0, 8, "Generated Summary from Transaction Agent:")
pdf.ln(4)

# Write the formatted content line by line
for line in lines:
    # Detect section headers and make them bold
    if re.match(r"^\d+\)|^###|^##|Summary|Flagged", line, re.IGNORECASE):
        pdf.set_font("Helvetica", style="B", size=12)
        pdf.multi_cell(0, 8, line)
        pdf.ln(2)
    # For bullet points, add indentation
    elif line.startswith("-"):
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(0, 7, "   " + line)
    # Normal lines
    else:
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(0, 7, line)
    pdf.ln(1)

# --- Export to memory and display a download link ---
# Instead of saving to disk, the PDF is written into memory (BytesIO buffer).
# A Base64-encoded link is created to let the user download the PDF directly in Colab.
buf = BytesIO()
pdf.output(buf)
pdf_bytes = buf.getvalue()

# Convert the PDF bytes to Base64 and display as an HTML download link
b64 = base64.b64encode(pdf_bytes).decode("ascii")
href = f'<a download="Transaction_Report.pdf" href="data:application/pdf;base64,{b64}" target="_blank">Download Transaction Report (PDF)</a>'
display(HTML(href))
